<h1>Endless Tic-Tac-Toe</h1>

The task is to realize endless version of Tic-Tac-Toe, but the player who succeeds in placing three of their marks, not five, in a diagonal, horizontal, or vertical row is the winner.

The main problem of realization is to avoid endless game field. My solution is to use list of non-empty cells. When player marks some cell, we can win only in square with size $9 \times 9$ and with this cell as a center.

If new marked cell has coordinates (x, y), program checks only square with lower left corner in (x - 5, y - 5) and upper right in (x + 5, y + 5).

Let's define list with tuples, everyone of them has three items - x, y and -1 or 1. If cell with coordinates (x, y) is not in this list, this cell is empty. 

This list is ordered by two coordinates. 

Firsly, we need to realize function, that gets cell from list of cells (if cell is not in list, it is empty and has value equal to 0). Next realization of standard tic-tac-toe follows with some changes. 
Call `main()` to start play.

In [1]:
from typing import List, Tuple, Union, Dict


def get_cell(point:Tuple[int], cells:Dict[Tuple, int]):
    if point in cells:
        return cells[point]
    else:
        return 0
    
    
def controller(player_to_move:int) -> str:
    '''
    Gets player's input.
    Arguments:
        player_to_move:int is equal to 1 or -1. 

    If player_to_move is equal to -1, string for input will be "Crosses' move (enter the position like 'x y' to set cross on field): "
    else if player_to_move is equal to 1, string for input will be "Zeros' move (enter the position like 'x y' to set zero on field): ".

    Returns players_input:str.
'''

    players_input:str = input("Zeros' move (enter the position like 'x y' to set cross on field): "
                              if player_to_move == 1 else
                              "Crosses' move (enter the position like 'x y' to set zero on field): " )
        
    return players_input  


def validate_input(players_input:str) -> int:
    '''
    Checks, is player's input is correct.

    Arguments:
        players_input:str;

    Returns:
    1 if input is correct else 0;
    '''

    # Note: correct input is string like 'x y' where x and y are numbers from [0, 2]

    # the alternative to this string is players_input.split()
    parts:List[str] = players_input.split(' ')

    # wrong format
    if len(parts) != 2:
        return 0

    # parts are not numbers
    try:
        x:float = float(parts[0])
        y:float = float(parts[1])
    except:
        return 0

    # parts are not whole numbers
    if (x % 1 != 0) or (y % 1 != 0):
        return 0 
    
    return 1
    
    
def split_to_numbers(players_input:str) -> Tuple[int]:
    '''
    Splits string like 'x y' to int(x), int(y).
    This function calls after validate_input().

    Arguments:
        players_input:str;
    Returns:
        Tuple[int]
    '''

    parts:List[str] = players_input.split(' ')

    x:int = int(parts[0])
    y:int = int(parts[1])

    return (x, y)


def get_winner_or_draw(point:Tuple[int], 
                       game_field:Dict[Tuple, int], 
                       player_to_move:int) -> int:
    '''
    Checks, is there winner in game for current move.

    Arguments:
        point:tuple - point to move;
        game_field:dict - dict with non-empty points;
        player_to_move:int - 1 or -1;
    Returns:
        0, if there is not winner in game;
        1, if crosses are winner;
        -1, if zeros are winner;
    ''' 
    
    x, y = point

    # check lines
    for i in range(x - 4, x + 1):
        cells_sum:int = 0;
        for j in range(i, i + 6):
            cells_sum += get_cell((j, y), game_field)
        
        if cells_sum == (5 * player_to_move):
            return player_to_move
        
    # check lines
    for i in range(y - 4, y + 1):
        cells_sum:int = 0 
        for j in range(i, i + 6):
            cells_sum += get_cell((x, j), game_field)
        
        if cells_sum == (5 * player_to_move):
            return player_to_move
        
    # check diagonals
    for i in range(x - 4, x + 1):
        cells_sum:int = 0
        for j in range(5):
            cells_sum += get_cell((i + j, i + j), game_field)
            
        if cells_sum == (5 * player_to_move):
            return player_to_move

    # there is not winner in game
    return 0


def model(players_input:str, 
          player_to_move:int,
          field_instance:Dict[Tuple, int],
          focus_point:Tuple[int]) -> Tuple[int, Dict[Tuple, int]]:
    '''
    Checks player's input, changes field if input is correct and simulates tick-tac-toe game.

    Arguments:
        players_input:str;
        player_to_move:int is -1 or 1;
        field_instance:np.ndarray is matrix with size (3, 3);

    Returns:
        is_finished:int is 0 or 1;
        input_was_correct:int is 0 or 1;
        winner:int is 0, -1 or 1;
        player_to_move:int is -1 or 1;
        field_instance:np.ndarray;
    '''

    if not validate_input(players_input):
        return (0, 0, 0, player_to_move, field_instance, focus_point)

    x, y = split_to_numbers(players_input)

    # field in input is not empty
    if get_cell((x, y), field_instance) != 0:
        return (0, 0, 0, player_to_move, field_instance, focus_point)

    # change field
    field_instance[(x, y)]:int = player_to_move

    # if winner is found
    move_result:int = get_winner_or_draw((x, y), 
                                         field_instance, 
                                         player_to_move)

    # winner was found
    # note: 0 - player_to_move means that other player takes turn
    if move_result != 0:
        return (1, 1, move_result, 0 - player_to_move, field_instance, (x, y))

    # continue game
    return (0, 1, 0, 0 - player_to_move, field_instance, (x, y))


def view(focus_point:Tuple[int],
         field_instance:Dict[Tuple, int]) -> None:
    '''
    Displays current instance of game field to screen.

    Arguments:
        field_instance:dict with point;
    '''

    cell_codes:Dict[int, str] = {0: ' ',
                                 1: 'o',
                                -1: 'x'}
        
    for y in range(focus_point[1] + 4, focus_point[1] - 5, -1):
        print('|', end=' ')
        
        for x in range(focus_point[0] - 4, focus_point[0] + 5):
            print(cell_codes[get_cell((x, y), field_instance)], end=' | ')
            
        print()


def game(player_to_move:int, 
         field_instance:Dict[Tuple, int],
         focus_point:Tuple[int]=(0, 0)) -> None:
    '''
    Simulates tick-tack-toe game recursively. 

    Arguments:
        players_to_move:int is equal to -1 or 1;
        field_instance:np.ndarray is matrix with size (3, 3);

    This function uses model(), which returns next fields:
        - is_finished:int;
        - input_was_correct:int;
        - winner:int;
        - player_to_move:int;
        - field_instance:np.ndarray;

    If is_finished is equal to 1, recursion stops.
    WARNING: this is very dangerous to make mistakes in input because 
             level of recursion in Python is not equal to infinity.
  '''
    game_results:Dict[int, str] = {0:"There is draw in this game!",
                                -1:"Crosses wins.",
                                 1:"Zeros wins."}

    players_input:str = controller(player_to_move)

    # model move
    (is_finished, 
     input_was_correct, 
     winner, 
     player_to_move, 
     field_instance,
     focus_point) = model(players_input, 
                          player_to_move, 
                          field_instance,
                          focus_point)

    # input is not correct
    if not input_was_correct:
        print('Invalid input. Please, try again.', end=' ')
    else:
        print('The game field now looks like: ')
        view(focus_point, field_instance)

    # winner was found
    if is_finished:
        print(game_results[winner])
    # winner was not found
    # continue game
    else:
        game(player_to_move, field_instance, focus_point)


def main() -> None:
    '''
    Starts game with default parameters. 
    Firstly crosses player takes turn and the field is empty.
    '''

    # set default parameters
    field_instance:Dict[Tuple, int] = {(0, 0):1,
                                       (1,1):1,
                                       (2, 2):1,
                                       (3, 3):1}
    player_to_move:int = -1

    game(player_to_move, field_instance)

In [ ]:
# start game
main()

Crosses' move (enter the position like 'x y' to set zero on field): 0 1
(-4, -4)
(-3, -3)
(-2, -2)
(-1, -1)
(0, 0)
1
(-3, -3)
(-2, -2)
(-1, -1)
(0, 0)
(1, 1)
2
(-2, -2)
(-1, -1)
(0, 0)
(1, 1)
(2, 2)
3
(-1, -1)
(0, 0)
(1, 1)
(2, 2)
(3, 3)
4
(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)
4
The game field now looks like: 
|   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   | o |   | 
|   |   |   |   |   |   | o |   |   | 
|   |   |   |   | x | o |   |   |   | 
|   |   |   |   | o |   |   |   |   | 
|   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   | 
Zeros' move (enter the position like 'x y' to set cross on field): 0 0
Invalid input. Please, try again. Zeros' move (enter the position like 'x y' to set cross on field): 1 1
Invalid input. Please, try again. 